In [1]:
## Data Collection
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

## Load the dataset
data= gutenberg.raw('shakespeare-hamlet.txt')
## save the file
with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/harishitalingam/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data pre-processing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## load the dataset
with open('hamlet.txt','r') as file:
    text = file.read().lower()

## Tokenize

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words 


4818

In [3]:
## creating the input sequience
inputsequences=[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        inputsequences.append(n_gram_sequence)

In [4]:
## pad sequence

max_sequence_len= max([len(x) for x in inputsequences])
max_sequence_len

14

In [5]:
inputsequences=np.array(pad_sequences(inputsequences,maxlen=max_sequence_len,padding='pre'))
inputsequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [6]:
## create predictors and lables
import tensorflow as tf
x,y = inputsequences[:,:-1],inputsequences[:,-1]

In [7]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
## split the dataset

x_train, x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [9]:
## Train LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

## Define model

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))
model.build(input_shape=(None, max_sequence_len-1))

## compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()


/Users/harishitalingam/miniconda3/envs/virtual_env1/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-10-16 13:34:58.014150: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-10-16 13:34:58.014175: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-10-16 13:34:58.014179: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-10-16 13:34:58.014195: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-16 13:34:58.014204: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bu

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
## train the model
history=model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100


2025-10-16 13:34:58.680388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 20ms/step - accuracy: 0.0334 - loss: 6.8828 - val_accuracy: 0.0342 - val_loss: 6.7472
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0396 - loss: 6.4462 - val_accuracy: 0.0427 - val_loss: 6.8349
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.0469 - loss: 6.3005 - val_accuracy: 0.0482 - val_loss: 6.8817
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0520 - loss: 6.1643 - val_accuracy: 0.0497 - val_loss: 6.9231
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0572 - loss: 6.0305 - val_accuracy: 0.0503 - val_loss: 6.9592
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.0641 - loss: 5.8872 - val_accuracy: 0.0589 - val_loss: 6.9943
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0732 - loss: 5.7387 - val_accuracy: 0.0649 - val_loss: 7.0496
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.0825 - loss: 5.5892 - val_